In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Setup
Pip install ultralytics and dependencies and check software and hardware.

In [ ]:
%pip install ultralytics
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

In [ ]:
pip install --force-reinstall numpy==1.23.0

# Train
From dataset

In [ ]:
!yolo train model="yolov8n.pt" data="/home/ubuntu/technoserve/new_dset/data.yaml" epochs=400 imgsz=1024 single_cls=True batch=64


In [ ]:
!yolo train model="yolov8n.pt" data="/home/ubuntu/technoserve/new_dset/data.yaml" epochs=100 imgsz=512 single_cls=True

# Test the model

In [ ]:
from IPython import display
display.display(display.Image("/mnt/rd/labeled/overripe/msg390418808-144197.jpg"))

In [ ]:
model = YOLO('runs/detect/train21/weights/best.pt')  # load a custom model
model.conf = 0.03

In [ ]:
# save as torchscript
model.export(format="torchscript")

In [ ]:
from PIL import Image
import torch
import torchvision.transforms as T
from torchvision import models
from torch import nn
# we need this function as it's being called (it's in the same file)
def xywh2xyxy(x):
    """
    Convert bounding box coordinates from (x, y, width, height) format to (x1, y1, x2, y2) format where (x1, y1) is the
    top-left corner and (x2, y2) is the bottom-right corner.

    Args:
        x (np.ndarray) or (torch.Tensor): The input bounding box coordinates in (x, y, width, height) format.
    Returns:
        y (np.ndarray) or (torch.Tensor): The bounding box coordinates in (x1, y1, x2, y2) format.
    """
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # top left x
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # top left y
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # bottom right x
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # bottom right y
    return y

class WrapperModel2(nn.Module):
  def __init__(self, model: torch.jit._script.RecursiveScriptModule):
    super().__init__()
    # model is the YOLO exported torchscript model
    self.model = model
  
  def forward(self, input_tensor: torch.Tensor, conf_thres:float=0.25):    
    multi_label=False
    max_time_img=0.05
    multi_label=False
    labels=()
    classes=None
    max_nms=30000
    max_wh=7680
    agnostic=False
    # conf_thres=0
    
    prediction = self.model(input_tensor)
    
    # Copy from /ultralytics/yolo/utils/ops.py non_max_suppression
    bs = prediction.shape[0]  # batch size
    nc = (prediction.shape[1] - 4)  # number of classes
    nm = prediction.shape[1] - nc - 4
    mi = 4 + nc  # mask start index
    xc = prediction[:, 4:mi].amax(1) > conf_thres
    
    # Settings
    # min_wh = 2  # (pixels) minimum box width and height
    
    time_limit = 0.5 + max_time_img * bs  # seconds to quit after
    redundant = True  # require redundant detections
    multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
    merge = False  # use merge-NMS
    
    # t = time.time()
    output = [torch.zeros((0, 6 + nm), device=prediction.device)] * bs
    for xi, x in enumerate(prediction):  # image index, image inference
        print(f"prediction: {xi}")
        # Apply constraints
        # x[((x[:, 2:4] < min_wh) | (x[:, 2:4] > max_wh)).any(1), 4] = 0  # width-height
        x = x.transpose(0, -1)[xc[xi]]  # confidence

        # Cat apriori labels if autolabelling
        # if labels and len(labels[xi]):
        #     lb = labels[xi]
        #     v = torch.zeros((len(lb), nc + nm + 5), device=x.device)
        #     v[:, :4] = lb[:, 1:5]  # box
        #     v[range(len(lb)), lb[:, 0].long() + 4] = 1.0  # cls
        #     x = torch.cat((x, v), 0)

        # If none remain process next image
        print(x.shape)
        if not x.shape[0]:
            continue

        # Detections matrix nx6 (xyxy, conf, cls)
        box, cls, mask = x.split((4, nc, nm), 1)
        box = xywh2xyxy(box)  # center_x, center_y, width, height) to (x1, y1, x2, y2)
        # if multi_label:
        #     i, j = (cls > conf_thres).nonzero(as_tuple=False).T
        #     x = torch.cat((box[i], x[i, 4 + j, None], j[:, None].float(), mask[i]), 1)
        # else:  # best class only
        #     conf, j = cls.max(1, keepdim=True)
        #     x = torch.cat((box, conf, j.float(), mask), 1)[conf.view(-1) > conf_thres]
            
        # * This is the "else" condition above
        conf, j = cls.max(1, keepdim=True)
        x = torch.cat((box, conf, j.float(), mask), 1)[conf.view(-1) > conf_thres]

        # Filter by class
        if classes is not None:
            x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

        # Apply finite constraint
        # if not torch.isfinite(x).all():
        #     x = x[torch.isfinite(x).all(1)]

        # Check shape
        n = x.shape[0]  # number of boxes
        if not n:  # no boxes
            continue
        x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence and remove excess boxes

        # Batched NMS
        c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
        boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores

        # forcefully return on the first prediction (which is OK for inference)
        return x , boxes, scores # actually only need x, while the others could be transformed in Android side from x

In [ ]:
# SAVE TORCH SCR

In [ ]:
torch_script_model = torch.jit.load("/home/ubuntu/technoserve/runs/detect/train21/weights/best.torchscript")

In [ ]:
import torchvision
w_model_2 = WrapperModel2(torch_script_model) # or in attempt 5, using WrapperModel(model)


In [ ]:
from torch.utils.mobile_optimizer import optimize_for_mobile
scripted_wrapped_model2 = torch.jit.script(w_model_2)
optimized_torchscript_b4_nms_model = optimize_for_mobile(scripted_wrapped_model2)
optimized_torchscript_b4_nms_model._save_for_lite_interpreter("./mobile_model_b4_nms.ptl")

In [ ]:
rslt = model("/mnt/rd/labeled/overripe/msg390418808-144197.jpg")#,predictor=ultralytics.engine.predictor.BasePredictor)

In [ ]:
rslt[0].boxes

In [ ]:



import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image = cv2.imread("/mnt/rd/labeled/overripe/msg5421324117-146123.jpg")

# Define the number of patches in each dimension
n_patches_h = 1  # number of patches vertically
n_patches_w = 1  # number of patches horizontally

# Calculate patch sizes
h, w, _ = image.shape
patch_h = h // n_patches_h
patch_w = w // n_patches_w

# Create patches
patches = []
for i in range(n_patches_h):
    for j in range(n_patches_w):
        patch = image[i*patch_h:(i+1)*patch_h, j*patch_w:(j+1)*patch_w]
        patches.append(patch)

# Detect objects in each patch
results = []
for patch in patches:
    results.append(model(patch)[0].plot(labels=False))

# Merge the results
merged_result = np.zeros((h, w, 3), dtype=np.uint8)
for i in range(n_patches_h):
    for j in range(n_patches_w):
        merged_result[i*patch_h:(i+1)*patch_h, j*patch_w:(j+1)*patch_w] = results[i*n_patches_w+j]

# Display the result
plt.figure(figsize=(6, 6))
plt.imshow((cv2.cvtColor(merged_result, cv2.COLOR_BGR2RGB)))
# plt.axis('off')
# plt.show()

In [ ]:
# Detect objects in each patch
results = []
detected_objects = []
for patch in patches:
    result = model(patch)[0]
    results.append(result.plot(labels=False))
    
    # Extract bounding boxes and crop objects
    for box in result.boxes.xyxy:
        x1, y1, x2, y2 = box.tolist()
        object_crop = patch[int(y1):int(y2), int(x1):int(x2)]
        detected_objects.append(object_crop)

# Function to resize and compute features
def compute_features(image, target_size=(64, 64)):
    # Resize the image
    resized = cv2.resize(image, target_size)
    
    # Compute color histogram
    hist = cv2.calcHist([resized], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist_feature = hist.flatten() / hist.sum()  # Normalize the histogram
    
    # Compute shape features
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt) / (target_size[0] * target_size[1])  # Normalized area
        perimeter = cv2.arcLength(cnt, True) / (2 * (target_size[0] + target_size[1]))  # Normalized perimeter
        shape_feature = [area, perimeter]
    else:
        shape_feature = [0, 0]
    
    # Combine features
    return np.concatenate([hist_feature, shape_feature])


In [ ]:

# Compute features for all detected objects
features = [compute_features(obj) for obj in detected_objects]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Perform K-means clustering
n_clusters = 4  # You can adjust this number
kmeans = KMeans(n_clusters=n_clusters)
labels = kmeans.fit_predict(features)


In [ ]:

# Assign colors to clusters
colors = [(255,0,0), (0,255,0), (0,0,255), (255, 255, 0)]  # RGB colors for each cluster

# Merge the results with cluster colors
merged_result = np.zeros((h, w, 3), dtype=np.uint8)
label_index = 0
for i, patch_result in enumerate(results):
    patch_with_labels = patch_result.copy()
    for box in model(patches[i])[0].boxes.xyxy:
        x1, y1, x2, y2 = box.tolist()
        color = colors[labels[label_index]]
        cv2.rectangle(patch_with_labels, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        label_index += 1
    
    i, j = divmod(i, n_patches_w)
    merged_result[i*patch_h:(i+1)*patch_h, j*patch_w:(j+1)*patch_w] = patch_with_labels

# Display the result
plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(merged_result, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
imageUrl = '/mnt/rd/labeled/overripe/msg390418808-144197.jpg'

# Load a model
#model = YOLO('yolov8n-seg.pt')  # load an official model

# Predict with the model
results = model(imageUrl)  # predict on an image

results_image = results[0].plot(labels=False)
plt.imshow(cv2.cvtColor(results_image, cv2.COLOR_BGR2RGB))

In [ ]:

from transformers import GLPNImageProcessor, GLPNForDepthEstimation
import torch
import numpy as np
from PIL import Image
import requests

image = Image.open("/mnt/rd/labeled/overripe/msg390418808-144140.jpg")

processor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-kitti")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-kitti")

# prepare image for the model
inputs = processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

# interpolate to original size
prediction = torch.nn.functional.interpolate(
    predicted_depth.unsqueeze(1),
    size=image.size[::-1],
    mode="bicubic",
    align_corners=False,
)

# visualize the prediction
output = prediction.squeeze().cpu().numpy()
formatted = (output * 255 / np.max(output)).astype("uint8")
depth = Image.fromarray(formatted)


In [ ]:
depth

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.cluster.hierarchy import linkage, fcluster
from matplotlib import pyplot as plt
def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

def adjust_gamma(img, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

def non_max_suppression_fast(boxes, scores, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1, y1, x2, y2 = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(scores)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return pick

def efficient_multi_scale_detection(image_path, model_path, conf_threshold=0.1, iou_threshold=0.3):
    model = YOLO(model_path)
    original_image = cv2.imread(image_path)
    h, w = original_image.shape[:2]

    scale_factors = [1.0, 1.25]
    patch_sizes = [(h//1, w//1), (h//2, w//2)]

    all_detections = []

    for scale in scale_factors:
        scaled_image = cv2.resize(original_image, (int(w*scale), int(h*scale)))
        
        enhanced_images = [
            scaled_image,
            apply_clahe(scaled_image),
            adjust_gamma(scaled_image, 0.8),
            adjust_gamma(scaled_image, 1.2)
        ]

        for img in enhanced_images:
            results = model(img, conf=conf_threshold)
            for r in results:
                boxes = r.boxes.xyxy.cpu().numpy()
                scores = r.boxes.conf.cpu().numpy()
                for box, score in zip(boxes, scores):
                    all_detections.append({
                        'box': box / scale,
                        'score': score,
                        'scale': scale
                    })

            for patch_h, patch_w in patch_sizes:
                for i in range(0, img.shape[0] - patch_h + 1, patch_h // 2):
                    for j in range(0, img.shape[1] - patch_w + 1, patch_w // 2):
                        patch = img[i:i+patch_h, j:j+patch_w]
                        results = model(patch, conf=conf_threshold)
                        for r in results:
                            boxes = r.boxes.xyxy.cpu().numpy()
                            scores = r.boxes.conf.cpu().numpy()
                            for box, score in zip(boxes, scores):
                                adjusted_box = [
                                    (box[0] + j) / scale,
                                    (box[1] + i) / scale,
                                    (box[2] + j) / scale,
                                    (box[3] + i) / scale
                                ]
                                all_detections.append({
                                    'box': adjusted_box,
                                    'score': score,
                                    'scale': scale
                                })

    boxes = np.array([d['box'] for d in all_detections])
    scores = np.array([d['score'] for d in all_detections])

    keep = non_max_suppression_fast(boxes, scores, iou_threshold)
    filtered_detections = [all_detections[i] for i in keep]

    detection_features = np.array([[d['box'][0], d['box'][1], d['box'][2], d['box'][3], d['score'], d['scale']] for d in filtered_detections])
    Z = linkage(detection_features, 'ward')
    clusters = fcluster(Z, t=0.5, criterion='distance')

    final_results = []
    for cluster_id in np.unique(clusters):
        cluster_detections = [filtered_detections[i] for i in range(len(filtered_detections)) if clusters[i] == cluster_id]
        best_detection = max(cluster_detections, key=lambda x: x['score'])
        final_results.append(best_detection)

    final_results.sort(key=lambda x: x['score'], reverse=True)

    return final_results, original_image

def merge_close_boundaries(detections, iou_threshold=0.7):
    merged = []
    detections.sort(key=lambda x: x['score'], reverse=True)
    
    for det in detections:
        if not merged:
            merged.append(det)
        else:
            should_merge = False
            for i, m_det in enumerate(merged):
                iou = calculate_iou(det['box'], m_det['box'])
                if iou > iou_threshold:
                    merged[i]['box'] = [
                        min(det['box'][0], m_det['box'][0]),
                        min(det['box'][1], m_det['box'][1]),
                        max(det['box'][2], m_det['box'][2]),
                        max(det['box'][3], m_det['box'][3])
                    ]
                    merged[i]['score'] = max(det['score'], m_det['score'])
                    should_merge = True
                    break
            if not should_merge:
                merged.append(det)
    
    return merged

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection)
    return iou



# Usage
# if __name__ == '__main__':
image_path = "/mnt/rd/labeled/overripe/msg390418808-144197.jpg"
model_path = 'runs/detect/train17/weights/best.pt'
detections, original_image = efficient_multi_scale_detection(image_path, model_path)

# Merge close boundaries
detections = merge_close_boundaries(detections)

# Visualize results
result_image = original_image.copy()
for det in detections:
    box = det['box']
    score = det['score']
    cv2.rectangle(result_image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
    # cv2.putText(result_image, f"{score:.2f}", (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

# plot the image
plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import time

def optimized_multi_scale_detection(image_path, model_path, conf_threshold=0.1, iou_threshold=0.5):
    def apply_clahe(img):
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl,a,b))
        return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    def non_max_suppression_fast(boxes, scores, overlapThresh):
        if len(boxes) == 0:
            return []

        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")

        pick = []

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(scores)

        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            overlap = (w * h) / area[idxs[:last]]

            idxs = np.delete(idxs, np.concatenate(([last],
                np.where(overlap > overlapThresh)[0])))

        return pick

    # Load the model
    model = YOLO(model_path)

    # Load and preprocess the image
    original_image = cv2.imread(image_path)
    h, w = original_image.shape[:2]

    # Define scale factors (reduced number of scales)
    scale_factors = [0.75, 1.0, 1.25]

    all_detections = []

    # Multi-scale detection
    for scale in scale_factors:
        scaled_image = cv2.resize(original_image, (int(w*scale), int(h*scale)))
        
        # Apply image enhancement
        enhanced_image = apply_clahe(scaled_image)

        # Whole image detection
        results = model(enhanced_image, conf=conf_threshold)
        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy()
            scores = r.boxes.conf.cpu().numpy()
            for box, score in zip(boxes, scores):
                all_detections.append({
                    'box': box / scale,
                    'score': score,
                    'scale': scale
                })

        # Adaptive patch-based detection
        if scale != 1.0:  # Only do patch-based detection for non-original scales
            patch_size = (h//3, w//3)
            stride = patch_size[0] // 2  # 50% overlap

            for i in range(0, scaled_image.shape[0] - patch_size[0] + 1, stride):
                for j in range(0, scaled_image.shape[1] - patch_size[1] + 1, stride):
                    patch = enhanced_image[i:i+patch_size[0], j:j+patch_size[1]]
                    results = model(patch, conf=conf_threshold)
                    for r in results:
                        boxes = r.boxes.xyxy.cpu().numpy()
                        scores = r.boxes.conf.cpu().numpy()
                        for box, score in zip(boxes, scores):
                            # Adjust coordinates to original image space
                            adjusted_box = [
                                (box[0] + j) / scale,
                                (box[1] + i) / scale,
                                (box[2] + j) / scale,
                                (box[3] + i) / scale
                            ]
                            all_detections.append({
                                'box': adjusted_box,
                                'score': score,
                                'scale': scale
                            })

    # Prepare data for non-max suppression
    boxes = np.array([d['box'] for d in all_detections])
    scores = np.array([d['score'] for d in all_detections])

    # Perform fast non-max suppression
    keep = non_max_suppression_fast(boxes, scores, iou_threshold)

    # Filter detections
    filtered_detections = [all_detections[i] for i in keep]

    # Sort final results by confidence score
    filtered_detections.sort(key=lambda x: x['score'], reverse=True)

    return filtered_detections, original_image

# Usage
start_time = time.time()

image_path = "/mnt/rd/labeled/ripe/msg6158697711-145910.jpg"
model_path = 'runs/detect/train17/weights/best.pt'
detections, original_image = optimized_multi_scale_detection(image_path, model_path)

end_time = time.time()
print(f"Detection time: {end_time - start_time:.2f} seconds")

# Visualize results
result_image = original_image.copy()
for det in detections:
    box = det['box']
    score = det['score']
    ccolor = (0, 255, 0) if score > 0.5 else ((255, 0, 0) if score < 0.2 else (0, 0, 255))
    cv2.rectangle(result_image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), ccolor, 2)
    # cv2.putText(result_image, f"{score:.2f}", (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
import cv2
import numpy as np

def generate_synthetic_image(size=(128, 128)):
    image = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    
    # Choose a random background type
    bg_type = np.random.choice(['solid', 'gradient', 'noise'])
    
    if bg_type == 'solid':
        # Solid color background
        color = np.random.randint(0, 256, 3)
        image[:] = color
    
    elif bg_type == 'gradient':
        # Gradient background
        start_color = np.random.randint(0, 256, 3)
        end_color = np.random.randint(0, 256, 3)
        for i in range(size[0]):
            ratio = i / size[0]
            color = start_color * (1 - ratio) + end_color * ratio
            image[i, :] = color
    
    else:  # noise
        # Colored noise background
        image = np.random.randint(0, 256, size=(size[0], size[1], 3), dtype=np.uint8)
    
    # Add some shapes
    num_shapes = np.random.randint(0, 6)
    for _ in range(num_shapes):
        shape_type = np.random.choice(['circle', 'rectangle', 'line'])
        color = np.random.randint(0, 256, 3).tolist()
        
        if shape_type == 'circle':
            center = tuple(np.random.randint(0, size[0], 2))
            radius = np.random.randint(10, 50)
            cv2.circle(image, center, radius, color, -1)
        
        elif shape_type == 'rectangle':
            pt1 = tuple(np.random.randint(0, size[0], 2))
            pt2 = tuple(np.random.randint(0, size[0], 2))
            cv2.rectangle(image, pt1, pt2, color, -1)
        
        else:  # line
            pt1 = tuple(np.random.randint(0, size[0], 2))
            pt2 = tuple(np.random.randint(0, size[0], 2))
            thickness = np.random.randint(1, 10)
            cv2.line(image, pt1, pt2, color, thickness)
    
    # Apply some blur to make it look more natural
    blur_amount = np.random.randint(0, 3) * 2 + 1
    image = cv2.GaussianBlur(image, (blur_amount, blur_amount), 0)
    
    # Randomly adjust brightness
    brightness = np.random.uniform(0.5, 1.5)
    image = np.clip(image * brightness, 0, 255).astype(np.uint8)
    
    return image

# You can test the function like this:
# test_image = generate_synthetic_image()
# cv2.imshow('Synthetic Image', test_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

import glob
import torch
import cv2
import random
import numpy as np
def yolo_to_x1y1x2y2(x_center, y_center, width, height, img_width, img_height):
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height
    
    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)
    
    return x1, y1, x2, y2
def pre_process_image(image, noise_up=False):
    if noise_up:
        # create noise
        if random.random() < 0.5:
            noise = np.random.normal(0, 255, image.shape) * 0.05
            image = image + noise
            image = np.clip(image, 0, 255)
            image = image.astype(np.uint8)
        else:

            # add brightness
            image = image + ((((random.random()*2)-1) * 0.05)*255)
            image = np.clip(image, 0, 255)
            # convert to int
            image = image.astype(np.uint8)

        if random.random() < 0.4:
            # split the image into half , 
            devisor_component = (random.random())+1
            if random.random() < 0.5:
                vertical = False
                if vertical:
                    middle_half = int(image.shape[0] / devisor_component)
                    image = image[:middle_half, :]
                else:
                    middle_half = int(image.shape[1] / devisor_component)
                    image = image[:, :middle_half]
            else:
                # split the image into half , 
                vertical = False
                if vertical:
                    middle_half = int(image.shape[0] / devisor_component)
                    image = image[middle_half:, :]
                else:
                    middle_half = int(image.shape[1] / devisor_component)
                    image = image[:, middle_half:]



    image = cv2.resize(image, (64, 64))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    
    image = image / 255.0
    image = np.transpose(image, (2, 0, 1))
    return image
class TrainingDataset(torch.utils.data.Dataset):
    def __init__(self, selection="train", data_loader_repeat=1):
        image_paths = glob.glob("/home/ubuntu/technoserve/neupw.v1i.yolov5pytorch/train/images/*.jpg")
        label_stores = glob.glob("/home/ubuntu/technoserve/neupw.v1i.yolov5pytorch/train/labels/*.txt")

        self.image_labels = {}

        for image_path in image_paths:
            image_id = image_path.split('/')[-1].split('.')[0]
            self.image_labels[image_path] = []
            # find from label stores
            for label_store in label_stores:
                label_id = label_store.split('/')[-1].split('.')[0]
                if label_id == image_id:
                    with open(label_store, 'r') as f:
                        for line in f:
                            classitem = line.strip().split(' ')
                            class_idx = int(classitem[0])
                            x_cent = float(classitem[1])
                            y_cent = float(classitem[2])
                            i_wid = float(classitem[3])
                            i_hei = float(classitem[4])
                            self.image_labels[image_path].append([class_idx, (x_cent, y_cent, i_wid, i_hei)])

        self.precompute_bounding_boxes()
        _class_image_pairs = []

        for image_path, labels in self.image_labels.items():
            # load image 
            image = cv2.imread(image_path)
            for label in labels:
                class_idx = label[0]
                # cut the image
                x_cent, y_cent, i_wid, y_wid = label[1]
                x1, y1, x2, y2 = yolo_to_x1y1x2y2(x_cent, y_cent, i_wid, y_wid, image.shape[1], image.shape[0])
                # print(x1, y1, x2, y2)
                crop_image = image[int(y1):int(y2), int(x1):int(x2)]
               
                
                _class_image_pairs.append((class_idx, crop_image))

            _negative_samples = self.find_negative_samples(image, image_path)
            self.negative_samples = _negative_samples
            print("added negative samples: ", len(_negative_samples))
            _class_image_pairs.extend(_negative_samples)
    
            
        self.data = _class_image_pairs

        # split into train and test
        train_size = int(0.85 * len(self.data))
        test_size = len(self.data) - train_size
        self.train_data, self.test_data = torch.utils.data.random_split(self.data, [train_size, test_size])
        self.noise_up  = data_loader_repeat > 1
        if selection == "train":
            self.data = list(self.train_data) * data_loader_repeat
        else:
            self.data =list( self.test_data)

    def precompute_bounding_boxes(self):
        self.bounding_boxes = {}
        for image_path, labels in self.image_labels.items():
            image = cv2.imread(image_path)
            height, width = image.shape[:2]
            boxes = []
            for label in labels:
                _, (x_cent, y_cent, i_wid, i_hei) = label
                bx1, by1, bx2, by2 = yolo_to_x1y1x2y2(x_cent, y_cent, i_wid, i_hei, width, height)
                boxes.append([bx1, by1, bx2, by2])
            self.bounding_boxes[image_path] = np.array(boxes)

    def find_negative_samples(self, image, image_path):
        negative_samples = []
        height, width = image.shape[:2]
        
        
        boxes = self.bounding_boxes[image_path]
        
        for _ in range(128):
            attempts = 0
            while attempts < 98:  # Limit attempts to avoid infinite loop
                crop_size = np.random.randint(32, 64)
                crop_ratio = np.random.uniform(0.7, 1.3)
                x1 = np.random.randint(0, width - (crop_size))
                y1 = np.random.randint(0, height - int(crop_size * (1 / crop_ratio)))
                x2, y2 = x1 + crop_size, y1 + int(crop_size * (1 / crop_ratio))
                
                # Vectorized overlap checking
                overlap = np.logical_and.reduce((
                    x1 < boxes[:, 2],
                    x2 > boxes[:, 0],
                    y1 < boxes[:, 3],
                    y2 > boxes[:, 1]
                ))
                
                if not np.any(overlap):
                    crop_image = image[y1:y2, x1:x2]
                    negative_samples.append((3, crop_image))
                    break
                
                attempts += 1
        
        return negative_samples

    def __len__(self):
        return int(len(self.data)*1.1)

    def __getitem__(self, idx):
        # if idx >= len(self.data):, then return synthetic image
        if idx >= len(self.data):
            data_itm = generate_synthetic_image()
            return pre_process_image(data_itm[1], self.noise_up), 3
        
        data_itm= (self.data)[idx]
        # print(data_itm[1].shape)
        return pre_process_image(data_itm[1], self.noise_up), data_itm[0]

In [ ]:
train_dataset = TrainingDataset("train", data_loader_repeat=10)
test_dataset = TrainingDataset("test")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# from matplotlib import pyplot as plt
# # train_dataset.negative_samples[0][1].shape
# plt.imshow(cv2.cvtColor(train_dataset.negative_samples[99][1], cv2.COLOR_BGR2RGB))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ClassifierModel(nn.Module):
    def __init__(self):
        super(ClassifierModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        
        self.dense_linears= nn.Sequential(
            nn.Linear(128 * 8 * 8, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 4)

        )
        

        self.dropout = nn.Dropout(0.1)

        # self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        # x shape: (B, 3, 128, 128)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)  # 64x64
        x = self.dropout(x)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)  # 32x32
        x = self.dropout(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)  # 32x32
        x = self.dropout(x)

        # x = F.relu(self.bn3(self.conv3(x)))
        # x = F.max_pool2d(x, 2)  # 32x32
        # x = self.dropout(x)

        # print ("x shape", x.shape)
        x = x.view(x.shape[0], -1)
        x = self.dense_linears(x)
        
        return x  # shape: (B, 3)

In [ ]:
init_model = ClassifierModel().cuda()
optimizer = torch.optim.Adam(init_model.parameters(), lr=0.0001)

In [ ]:
# torch.save(init_model.state_dict(), "/home/ubuntu/technoserve/classifier.pth")
# init_model.load_state_dict(torch.load("/home/ubuntu/technoserve/classifier.pth"))

In [ ]:
# torch script the model
init_model.eval()
from torch.utils.mobile_optimizer import optimize_for_mobile

scripted_module = torch.jit.script(init_model.cpu())
optimized_scripted_module = optimize_for_mobile(scripted_module)
optimized_scripted_module.save("./classifier.pt")

In [ ]:
import torch
import torch.nn.functional as F
from collections import defaultdict

# Define class weights
class_weights = torch.tensor([1.0, 1.0, 1.0, 0.4]).cuda()  # Less weight for "not-bean" class
major_penalty = 5.0  # Penalty multiplier for misclassifying beans as not-beans

for epoch in range(40):
    init_model.train()
    for i, (crop_image, class_idx) in enumerate(train_loader):
        crop_image = crop_image.detach().cuda().float()
        class_idx = class_idx.cuda()
        optimizer.zero_grad()
        output = init_model(crop_image)

        # print(output.shape, class_idx.shape, output.min(), output.max())
        
        # Calculate standard cross-entropy loss
        loss = F.cross_entropy(output, class_idx, weight=class_weights)
        
        # Add major penalty for misclassifying beans as not-beans
        _, predicted = torch.max(output, 1)
        bean_mask = class_idx != 3  # True for all bean classes
        misclassified_as_not_bean = (predicted == 3) & bean_mask
        penalty = major_penalty * misclassified_as_not_bean.float().mean()
        
        total_loss = loss + penalty
        
        total_loss.backward()
        optimizer.step()
        
        if i % 70 == 0:
            print(f"Loss: {loss.item():.4f}, Penalty: {penalty.item():.4f}, Total Loss: {total_loss.item():.4f}")
    
    print(f"Epoch {epoch+1} completed")
    
    # Test
    correct = 0
    total = 0
    class_correct = defaultdict(int)
    class_total = defaultdict(int)
    init_model.eval()
    
    with torch.no_grad():
        for crop_image, class_idx in test_loader:
            crop_image = crop_image.cuda().float()
            class_idx = class_idx.cuda()
            outputs = init_model(crop_image)
            _, predicted = torch.max(outputs.data, 1)
            total += class_idx.size(0)
            correct += (predicted == class_idx).sum().item()
            
            # Per-class accuracy
            for pred, true in zip(predicted, class_idx):
                if pred == true:
                    class_correct[true.item()] += 1
                class_total[true.item()] += 1

    print(f'Overall Accuracy of the network on the test images: {100 * correct / total:.2f}%')
    
    # Print per-class accuracy
    for class_idx in range(4):
        if class_total[class_idx] > 0:
            accuracy = 100 * class_correct[class_idx] / class_total[class_idx]
            print(f'Accuracy of class {class_idx}: {accuracy:.2f}%')
        else:
            print(f'No test samples for class {class_idx}')

    # Print the number of samples in each class
    for class_idx in range(4):
        print(f'Number of test samples in class {class_idx}: {class_total[class_idx]}')

    # Calculate and print false negative rate for beans
    false_negatives = sum(class_total[i] - class_correct[i] for i in range(3))  # Sum for all bean classes
    total_beans = sum(class_total[i] for i in range(3))
    false_negative_rate = 100 * false_negatives / total_beans if total_beans > 0 else 0
    print(f'False Negative Rate for Beans: {false_negative_rate:.2f}%')

In [ ]:
# import torch
# import torch.nn.functional as F
# from collections import defaultdict

# for epoch in range(10):
#     init_model.train()
#     for i, (crop_image, class_idx) in enumerate(train_loader):
#         crop_image = crop_image.cuda().float()
#         class_idx = class_idx.cuda()
#         optimizer.zero_grad()
#         output = init_model(crop_image)
#         loss = F.cross_entropy(output, class_idx)
#         loss.backward()
#         optimizer.step()
#         if i % 10 == 0:
#             print("Loss: ", loss.item())
#     print(f"Epoch {epoch+1} completed")
    
#     # Test
#     correct = 0
#     total = 0
#     class_correct = defaultdict(int)
#     class_total = defaultdict(int)
#     init_model.eval()
    
#     with torch.no_grad():
#         for crop_image, class_idx in test_loader:
#             crop_image = crop_image.cuda().float()
#             class_idx = class_idx.cuda()
#             outputs = init_model(crop_image)
#             _, predicted = torch.max(outputs.data, 1)
#             total += class_idx.size(0)
#             correct += (predicted == class_idx).sum().item()
            
#             # Per-class accuracy
#             for pred, true in zip(predicted, class_idx):
#                 if pred == true:
#                     class_correct[true.item()] += 1
#                 class_total[true.item()] += 1

#     print('Overall Accuracy of the network on the test images: %d %%' % (100 * correct / total))
    
#     # Print per-class accuracy
#     for class_idx in range(4):  # Assuming 3 classes (0, 1, 2)
#         if class_total[class_idx] > 0:
#             accuracy = 100 * class_correct[class_idx] / class_total[class_idx]
#             print(f'Accuracy of class {class_idx}: {accuracy:.2f}%')
#         else:
#             print(f'No test samples for class {class_idx}')

#     # Print the number of samples in each class
#     for class_idx in range(4):
#         print(f'Number of test samples in class {class_idx}: {class_total[class_idx]}')

In [ ]:
import logging
from ultralytics.utils import LOGGER
LOGGER.setLevel(logging.ERROR)

In [ ]:
%%time
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
from scipy.cluster.hierarchy import linkage, fcluster

def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

def adjust_gamma(img, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

def non_max_suppression_fast(boxes, scores, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1, y1, x2, y2 = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(scores)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return pick
 
def filter_detections_by_size(detections, std_threshold=3):
    # Calculate areas of all detections
    areas = [abs(d['box'][2] - d['box'][0]) * abs(d['box'][3] - d['box'][1]) for d in detections]
    print("non-filtered_detections:", len(detections))
    # Calculate mean and standard deviation of areas
    mean_area = np.mean(areas)
    std_area = np.std(areas)
    max_area = np.max(areas)
    min_area = np.min(areas)

    print("Mean area:", mean_area)
    print("Std area:", std_area)
    print("Max area:", max_area)
    print("Min area:", min_area)
    
    # Define acceptable range
    min_area = 200
    max_area = (mean_area*std_threshold)
    max_area = max(max_area, 20000)

    print("Min area:", min_area)
    print("Max area:", max_area)
    
    # Filter detections
    filtered_detections = [
        d for d, area in zip(detections, areas) 
        if min_area <= area <= max_area
    ]

    print("filtered_detections:", len(filtered_detections))
    
    return filtered_detections

def efficient_multi_scale_detection(image_path, model_path, conf_threshold=0.1, iou_threshold=0.3):
    model = YOLO(model_path)
    original_image = cv2.imread(image_path)
    h, w = original_image.shape[:2]

    scale_factors = [1.0, 1.5]
    patch_sizes = [(h//1, w//1), (h//2, w//2)]

    all_detections = []
    batch_inserts= []
    for scale in scale_factors:
        scaled_image = cv2.resize(original_image, (int(w*scale), int(h*scale)))
        
        enhanced_images = [
            scaled_image,
            apply_clahe(scaled_image),
            adjust_gamma(scaled_image, 0.8),
            adjust_gamma(scaled_image, 1.2)
        ]

        for img in enhanced_images:
            batch_inserts.append((img, scale, (0,0)))

            for patch_h, patch_w in patch_sizes:
                for i in range(0, img.shape[0] - patch_h + 1, patch_h // 2):
                    for j in range(0, img.shape[1] - patch_w + 1, patch_w // 2):
                        patch = img[i:i+patch_h, j:j+patch_w]
                        batch_inserts.append((patch, scale, (i, j)))



    results = model([v[0] for v in batch_inserts], conf=conf_threshold)
    for details, r in zip(batch_inserts,results):
        img, scale, (i, j) = details
        boxes = r.boxes.xyxy.cpu().numpy()
        scores = r.boxes.conf.cpu().numpy()
        for box, score in zip(boxes, scores):
            adjusted_box = [
                (box[0] + j) / scale,
                (box[1] + i) / scale,
                (box[2] + j) / scale,
                (box[3] + i) / scale
            ]
            all_detections.append({
                'box': adjusted_box,
                'score': score,
                'scale': scale
            })
            
    print("All detections:", len(all_detections))
    all_detections = filter_detections_by_size(all_detections)
    boxes = np.array([d['box'] for d in all_detections])
    scores = np.array([d['score'] for d in all_detections])

    
    keep = non_max_suppression_fast(boxes, scores, iou_threshold)
    
    filtered_detections = [all_detections[i] for i in keep]

    if len(filtered_detections) == 0:
        print("Nothing detected")
        return [], original_image

    detection_features = np.array([[d['box'][0], d['box'][1], d['box'][2], d['box'][3], d['score'], d['scale']] for d in filtered_detections])
    Z = linkage(detection_features, 'ward')
    # final_results = [all_detections[i] for i in keep]

    # final_results.sort(key=lambda x: x['score'], reverse=True)
    clusters = fcluster(Z, t=0.5, criterion='distance')

    final_results = []
    for cluster_id in np.unique(clusters):
        cluster_detections = [filtered_detections[i] for i in range(len(filtered_detections)) if clusters[i] == cluster_id]
        best_detection = max(cluster_detections, key=lambda x: x['score'])
        final_results.append(best_detection)

    final_results.sort(key=lambda x: x['score'], reverse=True)

    return final_results, original_image

def merge_close_boundaries(detections, iou_threshold=0.7):
    merged = []
    detections.sort(key=lambda x: x['score'], reverse=True)
    
    for det in detections:
        if not merged:
            merged.append(det)
        else:
            should_merge = False
            for i, m_det in enumerate(merged):
                iou = calculate_iou(det['box'], m_det['box'])
                if iou > iou_threshold:
                    merged[i]['box'] = [
                        min(det['box'][0], m_det['box'][0]),
                        min(det['box'][1], m_det['box'][1]),
                        max(det['box'][2], m_det['box'][2]),
                        max(det['box'][3], m_det['box'][3])
                    ]
                    merged[i]['score'] = max(det['score'], m_det['score'])
                    should_merge = True
                    break
            if not should_merge:
                merged.append(det)
    
    return merged

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection)
    return iou



# Usage
# if __name__ == '__main__':
image_path = "/home/ubuntu/technoserve/2024-07-05 11.28.08.jpg"
model_path = 'runs/detect/train17/weights/best.pt'
detections, original_image = efficient_multi_scale_detection(image_path, model_path)

# Merge close boundaries
detections = merge_close_boundaries(detections)
detections = filter_detections_by_size(detections)

# Visualize results
result_image = original_image.copy()
images = []
for det in detections:
    box = det['box']
    score = det['score']
    area = abs(box[2] - box[0]) * abs(box[3] - box[1])
    images.append((original_image[int(box[1]):int(box[3]), int(box[0]):int(box[2])], box, score))
    cv2.rectangle(result_image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
    # cv2.putText(result_image, f"{area:.2f}", (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
print ("Detection: ", len(images))
# plot the image
plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
images = [(pre_process_image(img, False), box, score)  for img, box, score in images]

In [ ]:
len(images)

In [ ]:
# Predict classes
init_model.eval()
batched_images = torch.stack([torch.Tensor(img) for img, _, _ in images])
with torch.no_grad():
    outputs = init_model(batched_images.cuda().float())
    # get variance of the output
    variance = torch.var(outputs, dim=1)
    _, predicted_clsx = torch.max(outputs.data, 1)


In [ ]:
predicted_clsx.shape

In [ ]:
# Visualize results
result_image = original_image.copy()
total_detection = len(images)
classes_count = [0, 0, 0, 0]
for i, (_, box, score) in enumerate(images):
    colrs = [
        (0, 255, 0),
        (255, 0, 0),
        (0, 0, 255),
        (0,0,0)
    ]
    # curr_score = (variance[i] - variance.min()) / (variance.max() - variance.min())
    sel_color = colrs[predicted_clsx[i].item()]
    classes_count[predicted_clsx[i].item()] += 1
    if not (score >= 0.05):
        sel_color = (0, 0, 0)
    cv2.rectangle(result_image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),sel_color , 2)

plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    

In [ ]:
for i in range(4):
    prcent = (classes_count[i] / total_detection) * 100
    class_namse = ["Overripe", "Ripe", "Underripe", "nan"]

    print(f"Class {class_namse[i]}: {prcent:.2f}%")

In [ ]:
# draw histogram of sizes of images by the class
import matplotlib.pyplot as plt
from collections import defaultdict

class_sizes = defaultdict(list)
max_width = 0
min_width = 100000
max_height = 0
min_height = 100000
for data in train_dataset:
    image, class_idx = data
    class_sizes[class_idx].append(image.shape[0] * image.shape[1])
    max_width = max(max_width, image.shape[1])
    min_width = min(min_width, image.shape[1])
    max_height = max(max_height, image.shape[0])
    min_height = min(min_height, image.shape[0])
    

plt.figure(figsize=(12, 6))
plt.hist([size for size in class_sizes.values()], bins=50, label=[f"Class {class_idx}" for class_idx in class_sizes])
plt.xlabel("Image Size")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In [ ]:
import glob
import os
import cv2
import tqdm
from multiprocessing import Pool
from PIL import Image
imgfls = glob.glob("/mnt/rd/celebpic/pictures/**/*.jpg")

# multiprocessing pool to check if the images are valid , else delete them
def check_image(imgfl):
    try:
        img = Image.open(imgfl)
        img.verify()
        img.convert("RGB")
        
    except:
        os.remove(imgfl)

with Pool(24) as p:
    list(tqdm.tqdm(p.imap(check_image, imgfls), total=len(imgfls)))
